In [ ]:
import json
import xmltodict

# Otevře xml soubor do proměnné f a metodou parse načte proměnnou f do data.
# xml je slovník s dalšími slovníky. 
with open("ZDROJ_Rejstrik_skol.xml", "rb") as f:                    
    data = xmltodict.parse(f)

print(type(data))                                             
print(data.keys())                                            
export = data["ExportDat"]                                    
print(type(export))                                           
for key in export.keys():                                    
    print(key, type(export[key]), len(export[key]))           

In [3]:
# převod do JSONu, ve kterém je vidět struktura dat a je přehlednější pro pochopení další práce.
with open("JSON_Rejstrik_skol.json", "w", encoding="utf8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)     

In [ ]:
# rozklíčování xml souboru a vypsání pouze těch sloupců, které chceme do seznamu zaznamy
import csv
from collections import Counter

c = Counter()
subjekty = data["ExportDat"]["PravniSubjekt"]                               
zaznamy = []                                                                

for subj in subjekty:                                                                                                         
    for key, detaily_subjektu in subj.items():                                
        c[key] += 1                                                           
        if key == "SkolyZarizeni":                                            
            skolska_zarizeni = detaily_subjektu["SkolaZarizeni"]              

            c[f"{key}.SkolaZarizeni.{type(skolska_zarizeni)}"] += 1           
                                                                             

            if type(skolska_zarizeni) == list:                                
                c[f"{key}.SkolaZarizeni"] += len(skolska_zarizeni)            
            else:
                c[f"{key}.SkolaZarizeni"] += 1                                

            if type(skolska_zarizeni) == dict:                                
                skolska_zarizeni = [detaily_subjektu["SkolaZarizeni"]]        

            strednich_skol = 0

            for zarizeni in skolska_zarizeni:
                # vypisujeme pouze ze středních škol = "C00"
                if zarizeni["SkolaDruhTyp"] != "C00":                           
                    continue                                                    

                strednich_skol += 1                                             

                c[f"{key}.SkolyZarizeni.{zarizeni['SkolaDruhTyp']}"] += 1
                # ošetření vyjímek
                try:                                                            
                    kapacita = int(zarizeni["SkolaKapacita"])                   
                    c[f"{key}.SkolaZarizeni.kapacita"] += kapacita
                except:
                    kapacita = 0                                               
                    c[f"{key}.SkolaZarizeni.kapacita.{zarizeni['SkolaKapacita']}"] += 1 

                # spojení sloupců adres pro kompletní adresu       
                adresa = str(subj["Reditelstvi"]["RedAdresa1"]) + ", " + str(subj["Reditelstvi"]["RedAdresa2"])

                zaznamy.append(                                                 
                    {                                                           
                        "RedIzo": subj["RedIzo"],                               
                        "Název školy": subj["Reditelstvi"]["RedPlnyNazev"],
                        "Adresa 1": subj["Reditelstvi"]["RedAdresa1"],
                        "Adresa 2": subj["Reditelstvi"]["RedAdresa2"],
                        "Adresa 3": subj["Reditelstvi"]["RedAdresa3"],
                        "ADRESA_KOMPLET": adresa,
                        "Okres": subj["Reditelstvi"]["Okres"],
                        "Právní forma": subj["Reditelstvi"]["PravniForma"],
                        "Zřizovatel": subj["Reditelstvi"]["DruhZrizovatele"],
                        "Název zařízení": zarizeni["SkolaPlnyNazev"],
                        "IZO": zarizeni["IZO"],
                        "Kapacita zařízení": kapacita,
                    }
                )

            c[f"{key}.SkolaZarizeni.strednich_skol={strednich_skol}"] += 1



if not zaznamy:
    exit()

# uložení výsledné tabulky do csv
with open("CSV_Rejstrik_skol.csv", "w", newline="", encoding="utf8") as csvfile:
    fieldnames = zaznamy[0].keys()                                               
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)                      

    writer.writeheader()                                                        
    for zaznam in zaznamy:
        writer.writerow(zaznam)

sorted(c.items())                                                               
